# Catastro

In [0]:
# listado de ubicacion shape de Lotes de bogota con geometria de punto.
[a.path for a in dbutils.fs.ls("/FileStore/tables/Catastro/2023_octubre")]
#dbfs:/FileStore/
#dbfs:/FileStore/

In [0]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from sedona.spark import *
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

from sedona.register.geo_registrator import SedonaRegistrator
# configuración de spark y Sedona para datos geograficos
spark = SparkSession.\
    builder.\
    master("local[*]").\
    appName("Sedona App").\
    config("spark.serializer", KryoSerializer.getName).\
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName) .\
    config("spark.kryoserializer.buffer.max", "200gb").\
    getOrCreate()

#SedonaRegistrator.registerAll()
SedonaContext.create(spark)

In [0]:
#import geopandas as gpd
#lote = gpd.read_file("/dbfs/FileStore/tables/shp/Lotes_centroide/Puntos_lote.shp")

# LOTES FILTRADOS PLMB
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.utils.adapter import Adapter
sc = spark.sparkContext
sc.setSystemProperty("sedona.global.charset", "utf8")
#Lotes
Lotes = ShapefileReader.readToGeometryRDD(sc, "dbfs:/FileStore/tables/shp/Lotes_centroide/")
Isocrona = ShapefileReader.readToGeometryRDD(sc, "dbfs:/FileStore/tables/shp/isocronas/")
Lotes_df = Adapter.toDf(Lotes, spark)
Isocrona_df = Adapter.toDf(Isocrona, spark)

In [0]:
Lotes.CRSTransform('epsg:4326','epsg:4326')
Isocrona.CRSTransform('epsg:4326','epsg:4326')

In [0]:
Lotes.rawSpatialRDD.filter(lambda x: [x.userData.split("\t")]).collect()

In [0]:
import geopandas as gpd
l_gdf = gpd.GeoDataFrame(
    Lotes.rawSpatialRDD.map(lambda x: [x.geom]).sample(False, 0.0009,6).collect(),
    columns=["geom"],
    geometry="geom",
    crs = 'epsg:4326'
)

In [0]:
from sedona.core.spatialOperator import JoinQuery
# haciendo las particiones espaciales
Lotes.analyze()
Lotes.spatialPartitioning(GridType.KDBTREE, 4)
Isocrona.spatialPartitioning(Lotes.getPartitioner())

In [0]:
from sedona.core.enums import GridType
from sedona.core.enums import IndexType
from sedona.core.spatialOperator import JoinQuery


build_on_spatial_partitioned_rdd = True ## Set to TRUE only if run join query
using_index = True
Isocrona.buildIndex(IndexType.QUADTREE, build_on_spatial_partitioned_rdd)
# union y selección de las entidades espaciales dentro de la isocrona
result = JoinQuery.SpatialJoinQueryFlat(Lotes, Isocrona, using_index, True)

In [0]:
import geopandas as gpd
valores = gpd.GeoDataFrame(
    result.map(lambda x: [x[1].geom, *x[0].userData.split("\t"), *x[1].userData.split("\t")]).collect(),columns=["geom", 'group_inde','Tiempo','layer','Linea','LOTSECT_ID','LOTMANZ_ID','LOTLOTE_ID','LOTZHF_ID','LOTZHG_ID','LOTUNIDAPH','LOTDISTRIT','LOTLSIMBOL','ESTADO_REG','FECHA_REGI','FECHA_DESD','FECHA_HAST','LOTLNUMERO','FRENTE','FONDO'],
    geometry="geom",
    crs= "epsg:4326"
)
#.sample(False, 0.0009,6)

In [0]:
import numpy as np
valores['Linea'].value_counts()

In [0]:
valores[['geom','Tiempo']].sample(2000).explore('Tiempo')

# CRUCE CON USOS E INMUEBLES.

In [0]:
import os
startpath = '/dbfs/FileStore/tables/Catastro/'

# crea listados por grupo de archivos para cada año junto a un diccionario de la información disponible de catastro
# desde el año 2006 hasta 2022
ph = list()
pre = list()
uso = list()
dict_anio = dict()

for root, dirs, files in os.walk(startpath):
    periodos = [str(a) for a in range(2006,2023)]
    # recorre el arreglo de años   
    if os.path.basename(root) in periodos:
        carpeta = os.path.basename(root)
        ph_predios = '{}/ph_{}.csv'.format(root, carpeta)
        predios = '{}/predios_{}.csv'.format(root, carpeta)
        usos = '{}/usos_{}.csv'.format(root, carpeta)
        lista_interna = list()
        for f in files:
            if 'ph_' in f:
                lista_interna.append(ph_predios)
                ph.append(ph_predios)
            elif 'predios_'in f:
                lista_interna.append(predios)
                pre.append(predios)
            elif 'usos_'in f:
                lista_interna.append(usos)
                uso.append(usos)
            else:
                pass
        dict_anio[carpeta] = lista_interna
        del lista_interna
    else:
        pass

In [0]:
# se definen los usos definidos y agrupados por los profesionales del equipo OOVS
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Iniciar una sesión de Spark
spark = SparkSession.builder.appName("usos").getOrCreate()

hUsos={
    'Asociar al uso principal':['AREA DE MEZANINE EN PH','DEPOSITO (LOCKERS) PH','DEPOSITO ALMACENAMIENTO PH','PARQUEO CUBIERTO NPH','PARQUEO CUBIERTO PH','PARQUEO LIBRE PH','PISCINAS EN NPH','PISCINAS EN PH','SECADEROS'],
    'Comercio y servicios':['BODEGA COMERCIAL NPH','BODEGA COMERCIAL PH','BODEGA ECONOMICA','BODEGA ECONOMICA(SERVITECA,ESTA.SERVIC.)','BODEGAS DE ALMACENAMIENTO NPH','BODEGAS DE ALMACENAMIENTO PH','CENTRO COMERCIAL GRANDE NPH','CENTRO COMERCIAL GRANDE PH','CENTRO COMERCIAL MEDIANO NPH','CENTRO COMERCIAL MEDIANO PH','CENTRO COMERCIAL PEQUENO NPH','CENTRO COMERCIAL PEQUENO PH','CLUBES PEQUENOS','COMERCIO PUNTUAL NPH O HASTA 3 UNID PH','COMERCIO PUNTUAL PH','CORREDOR COMERCIAL NPH O HASTA 3 UNID PH','CORREDOR COMERCIAL PH','DEPOSITOS DE ALMACENAMIENTO NPH','EDIFICIOS DE PARQUEO NPH','EDIFICIOS DE PARQUEO PH','HOTELES NPH','HOTELES PH','MOTELES, AMOBLADOS, RESIDENCIAS NPH','MOTELES, AMOBLADOS, RESIDENCIAS PH','OFICINA BODEGA Y/O INDUSTRIA PH','OFICINAS EN BODEGAS Y/O INDUSTRIAS','OFICINAS OPERATIVAS','OFICINAS OPERATIVAS(ESTACIONES SERVICIO)','OFICINAS Y CONSULTORIOS NPH','OFICINAS Y CONSULTORIOS PH','PARQUES DE DIVERSION','RESTAURANTES NPH','RESTAURANTES PH','TEATROS Y CINEMAS NPH','TEATROS Y CINEMAS PH', 'PARQUES DE DIVERSION EN P.H.'],
    'Dotacional':['AULAS DE CLASE','CEMENTERIOS','CENTROS MEDICOS EN PH','CLINICAS, HOSPITALES, CENTROS MEDICOS','CLUBES MAYOR EXTENSION','COLEGIOS EN PH','COLEGIOS Y UNIVERSIDADES 1 A 3 PISOS','COLEGIOS Y UNIVERSIDADES 4 O MAS PISOS','COLISEOS','CULTO RELIGIOS EN NPH','CULTO RELIGIOSO EN PH','IGLESIA PH','IGLESIAS','INSTALACIONES MILITARES','INSTITUCIONAL PH','INSTITUCIONAL PUNTUAL','MUSEOS','OFICINAS Y CONSULTORIOS (OFICIAL) NPH','OFICINAS Y CONSULTORIOS (OFICIAL) PH','PLAZAS DE MERCADO'],
    'Industrial':['INDUSTRIA ARTESANAL','INDUSTRIA GRANDE','INDUSTRIA GRANDE PH','INDUSTRIA MEDIANA','INDUSTRIA MEDIANA PH'],
    'Otro':['COCHERAS, MARRANERAS, PORQUERIZAS','ENRAMADAS, COBERTIZOS, CANEYES','ESTABLOS, PESEBRERAS','GALPONES, GALLINEROS','KIOSKOS','LOTE EN PROPIEDAD HORIZONTAL','PISTA AEROPUERTO','SILOS'],
    'Residencial':['HABITACIONAL EN PROPIEDAD HORIZONTAL','HABITACIONAL MAYOR O IGUAL A 4 PISOS NPH O 3 PISOS PH','HABITACIONAL MENOR O IGUAL A 3 PISOS NPH','HABITACIONAL MENOR O IGUAL A 3 PISOS PH','MAYOR O IGUAL A 4 PISOS NPH O 3 PISOS PH']
 }

def homoUso(uso_catastro:str,usos_agregados=hUsos):
    '''
    Toma el uso de la base de predios de catastro y lo agrega a 
    categorias mas sencillas de entender para el análisis de datos del OOVS

    Args:
        input_str (str): Cadena de texto que trae el uso para gregarlo.

    Returns:
        str: retorna el uso agragado y homologado por el equipo del observatorio
    '''
    for k,v in usos_agregados.items():
        if uso_catastro in v:
            return k
        elif uso_catastro not in v:
            pass

# Registrar la función Python como UDF de Spark
homoUso_udf = udf(homoUso, StringType())

In [0]:
# recorre el arreglo de los archivos de uso y crea un nuevo campo con el uso agregado, luego lo guarda en un archivo nuevo.
for u in uso:
    path = u.replace("/dbfs","dbfs:")
    uso_pdf = spark.read.format("csv").option("header", "true").load(path)
    df = uso_pdf.withColumn("UsoAgregado", homoUso_udf(uso_pdf["DESCRIPCION_USO"]))
    new_name = path.replace('.csv','_agregado.csv') #uso homologado
    df.write.csv(new_name, header=True, mode="overwrite")

In [0]:
[a.path for a in dbutils.fs.ls("/FileStore/tables/Catastro/2010/")]

In [0]:

uso_pdf = spark.read.format("csv").option("header", "true").option("delimiter", ";").load('dbfs:/FileStore/tables/Catastro/2023_octubre/usos2023.csv')
df = uso_pdf.withColumn("UsoAgregado", homoUso_udf(uso_pdf["DESCRIPCION_USO"]))
#new_name = path.replace('.csv','_agregado.csv') #uso homologado
#df.write.csv('dbfs:/FileStore/tables/Catastro/2023_octubre/usos_2023_agregado.csv', header=True, mode="overwrite")

In [0]:
sum(df.toPandas().UsoAgregado.isnull())
#value_counts()

In [0]:
# Lotes
l2023 = IdLote(pdf)

In [0]:
l2023_pdf = spark.createDataFrame(l2023)

In [0]:
intersection_predios = df.join(l2023_pdf, 'CHIP', "inner")

In [0]:

inter=intersection_predios.groupBy("LOTLOTE_ID").pivot("UsoAgregado").count()

In [0]:
inter.display()

# Conteo usos por lote

In [0]:
from pyspark.sql.functions import sum
uA = spark.read.format("csv").option("header", "true").load('dbfs:/FileStore/tables/Catastro/2010/usos_2010_agregado.csv/')

In [0]:
#uA.groupBy("LOTLOTE_ID").count().show()

conteoUsos = uA.groupBy("LOTLOTE_ID").pivot("UsoAgregado").count()
#uA.select('DESCRIPCION_USO').where(uA.UsoAgregado.isNull()).groupBy("DESCRIPCION_USO").count().display()

In [0]:
conteoUsos.display()

In [0]:
pred_example = spark.read.format("csv").option("header", "true").load('dbfs:/FileStore/tables/Catastro/2010/predios_2010.csv/')

In [0]:
#pred_example.groupBy("LOTLOTE_ID").count().display()
agrupacion_predios = pred_example.groupBy("LOTLOTE_ID").agg({"AREA_TERRENO":"sum", "AREA_CONSTRUIDA":"sum", "VALOR_AVALUO":"sum", "CHIP":"count"})

agrupacion_predios = agrupacion_predios.withColumnRenamed("count(CHIP)","NumPredios") \
    .withColumnRenamed("sum(VALOR_AVALUO)","ValorAvaluo") \
    .withColumnRenamed("sum(AREA_CONSTRUIDA)","AC") \
    .withColumnRenamed("sum(AREA_TERRENO)","AT")

union = agrupacion_predios.join(conteoUsos, 'LOTLOTE_ID', "inner")

In [0]:
union.display()

In [0]:
ph_proce = spark.read.format("csv").option("header", "true").load('dbfs:/FileStore/tables/Catastro/2010/ph_2010.csv')
ph_proce = ph_proce.select("LOTLOTE_ID","NUMERO_UNIDADES_PH")
ph_proce = ph_proce.withColumnRenamed("NUMERO_UNIDADES_PH","Unidades_En_PH")

union2 = union.join(ph_proce, 'LOTLOTE_ID')
# este dato no tiene sentido cruzar solo los predios o lotes que tienen PH

In [0]:
union2.display()


### revision de valores 2023

In [0]:
import pandas as pd
#df = pd.read_csv('/dbfs/FileStore/tables/Catastro/2023_octubre/SOL0285553_23.txt', sep=';')

In [0]:
df1 = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("dbfs:/FileStore/tables/Catastro/2023_octubre/SOL0285553_23.csv")

In [0]:
import pandas as pd
import geopandas as gpd

pdf = df1.toPandas()
pdf['COORDENADA_X'] = pdf['COORDENADA_X'].apply(lambda x: x.replace(',','.')).astype('double')
pdf['COORDENADA_Y'] = pdf['COORDENADA_Y'].apply(lambda x: x.replace(',','.')).astype('double')
pdf = gpd.GeoDataFrame(pdf, geometry = gpd.points_from_xy(pdf['COORDENADA_X'].replace(',','.'), pdf['COORDENADA_Y'].replace(',','.')), crs= 'esri:102233')

In [0]:
def IdLote(dataset):
    dataset['CODIGO_BARRIO'] = dataset['CODIGO_BARRIO'].astype(str)
    dataset['CODIGO_MANZANA'] = dataset['CODIGO_MANZANA'].astype(str)
    dataset['CODIGO_PREDIO'] = dataset['CODIGO_PREDIO'].astype(str)

    for index, row in dataset.iterrows():
        if len(row['CODIGO_BARRIO']) == 4:
            dataset.at[index,'CODIGO_BARRIO'] = '00{}'.format(row['CODIGO_BARRIO'])
        else:
            pass

        if len(row['CODIGO_MANZANA']) == 1:
            dataset.at[index,'CODIGO_MANZANA'] = '00{}'.format(row['CODIGO_MANZANA'])
        elif len(row['CODIGO_MANZANA']) == 2:
            dataset.at[index,'CODIGO_MANZANA'] = '0{}'.format(row['CODIGO_MANZANA'])
        else:
            pass

        if len(row['CODIGO_PREDIO']) == 1:
            dataset.at[index,'CODIGO_PREDIO'] = '00{}'.format(row['CODIGO_PREDIO'])
        elif len(row['CODIGO_PREDIO']) == 2:
            dataset.at[index,'CODIGO_PREDIO'] = '0{}'.format(row['CODIGO_PREDIO'])
        else:
            pass

    dataset['LOTLOTE_ID'] = (dataset['CODIGO_BARRIO'].astype(str) + dataset['CODIGO_MANZANA'].astype(str) + dataset['CODIGO_PREDIO'].astype(str)).astype(str)
    return dataset

In [0]:
pdf['AREA_CONSTRUIDA'] = pdf.AREA_CONSTRUIDA.fillna(0)
pdf['AREA_CONSTRUIDA'] = pdf['AREA_CONSTRUIDA'].astype(str).apply(lambda p: p.replace(",",'.')).astype(float)
pdf['AREA_TERRENO'] = pdf['AREA_TERRENO'].astype(str).apply(lambda p: p.replace(",",'.')).astype(float)
pdf['VALOR_AVALUO'] = pdf['VALOR_AVALUO'].astype(str).apply(lambda p: p.replace(",",'.')).astype(float)

pdf['AT_VM2'] =  pdf['VALOR_AVALUO'] / pdf['AREA_TERRENO']
pdf['AC_VM2'] =  pdf['VALOR_AVALUO'] / pdf['AREA_CONSTRUIDA']

In [0]:
import numpy as np
predios = IdLote(pdf)

table_pivot = pd.pivot_table(predios, values=['AT_VM2', 'AC_VM2', 'AREA_TERRENO', 'AREA_CONSTRUIDA'],
                            index=['LOTLOTE_ID'], aggfunc={'AT_VM2': np.mean, 'AC_VM2': np.mean, 'AREA_TERRENO':np.sum, 'AREA_CONSTRUIDA':np.sum})
                            
table_pivot.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [0]:
import geopandas as gpd
MetroL = gpd.read_file('/dbfs/FileStore/tables/shp/Lotes_centroide/Puntos_lote.shp')

result = pd.merge(MetroL,table_pivot, how='inner',left_on=['LOTLOTE_ID'],right_on=['LOTLOTE_ID'])

In [0]:
union = gpd.GeoDataFrame(result, geometry=result.geometry, crs='4326')

In [0]:
iso = gpd.read_file('/dbfs/FileStore/tables/shp/isocronas/Isocrona_por_estacion_TOTAL_2022.shp')

In [0]:
datos_relacionados = gpd.sjoin(union.to_crs('EPSG:4326'), iso.set_crs('EPSG:4326'), how='inner', predicate='intersects')

In [0]:
datos_relacionados.Tiempo.value_counts()

In [0]:
datos_relacionados.AT_VM2.isin([np.inf, -np.inf]).sum()

In [0]:
datos_relacionados = datos_relacionados.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [0]:
datos_relacionados.groupby(by=['Tiempo'])['AC_VM2','AT_VM2'].mean()

In [0]:
#datos_relacionados.groupby(by = 'Tiempo').agg({'AC_VM2':np.mean})
pd.set_option('display.float_format', '{.1f}'.format)

datos_relacionados.groupby(by=['Tiempo'])['AC_VM2','AT_VM2'].mean()

# Solicitud GAA

In [0]:
n = list()
dictToReplace ={-1:'0',-2:'00',-3:'000',-4:'0000'}
for i in lista_consulta:
    if '-' in i:
        partes=i.split('-')
        circulo = f'0{partes[0]}' if partes[0][0]!='0' else partes[0]
        if len(partes[1])!=8:
            adicion = dictToReplace[len(partes[1])-8]
            #print(adicion)
            complemento = f'{adicion}{partes[1]}'
            folio = circulo+complemento
            n.append(folio)
        else:
            folio = circulo+partes[1]
            n.append(folio)
    else:
        n.append(folio)
        
        #3complemento = 

        #n.append(f'0{i}'.replace("-",""))
    #n.append(i.replace("-",""))
#new_list
#pdf.tail(50)

consulta = pdf.query('MATRICULA in @n')

In [0]:
consulta = pdf.query('CHIP in @l_chips')

In [0]:
len(l_chips)

In [0]:
consulta.explore()

In [0]:
consulta.columns

In [0]:
resultado = consulta[['MATRICULA','CHIP','VALOR_AVALUO','geometry']].rename(columns={'VALOR_AVALUO':'VALOR_AVALUO_2023'}).reset_index().drop(['index'],axis=1)
#resultado = resultado.drop(resultado[resultado.MATRICULA=='050S00022547'].index)

In [0]:
resultado.shape

In [0]:
resultado.explore()

In [0]:
len(lista_consulta)-len(resultado.MATRICULA.to_list())

In [0]:
def freverse(matricula,lista_consulta=lista_consulta):
    for i in lista_consulta:
        if '-' in i:
            partes=i.split('-')
            circulo = f'0{partes[0]}' if partes[0][0]!='0' else partes[0]
            if len(partes[1])!=8:
                adicion = dictToReplace[len(partes[1])-8]
                #print(adicion)
                complemento = f'{adicion}{partes[1]}'
                folio = circulo+complemento
                if matricula==folio:
                    return i
            else:
                folio = circulo+partes[1]
                if matricula==folio:
                    return i

In [0]:
resultado['MATRICULA_SUP'] = resultado['MATRICULA'].apply(lambda x: freverse(x))

In [0]:
resultado.to_csv('/dbfs/FileStore/tables/Catastro/2023_octubre/respuesta_predios_OOVS.csv', encoding='utf-8')

In [0]:
%python

displayHTML("""<a href="https://adb-5010697992891525.5.azuredatabricks.net/files/tables/Catastro/2023_octubre/respuesta_predios_OOVS.csv?o=5010697992891525" download>Download csv </a>""")

In [0]:
l_chips = ['AAA0149DEKC',
'AAA0149DEPA',
'AAA0149DERJ',
'AAA0149DEYN',
'AAA0149DEXS',
'AAA0149DEWW',
'AAA0148ZAUZ',
'AAA0148ZAYX',
'AAA0148ZAZM',
'AAA0148ZBAW',
'AAA0148ZBBS',
'AAA0148ZBHK',
'AAA0148ZBFZ',
'AAA0148ZBEP',
'AAA0148ZBCN',
'AAA0148ZUSK',
'AAA0148ZUTO',
'AAA0148ZUWF',
'AAA0148ZWEA',
'AAA0148ZWDM',
'AAA0148ZWCX',
'AAA0148ZWBR',
'AAA0148ZWAF',
'AAA0148ZUYX',
'AAA0149ADSY',
'AAA0149ADNX',
'AAA0149ADOM',
'AAA0149DFSK',
'AAA0149DFTO',
'AAA0149DFUZ',
'AAA0149DFWF',
'AAA0149DFXR',
'AAA0149DFYX',
'AAA0149DFOE',
'AAA0149DHWF',
'AAA0149DHUZ',
'AAA0149DHSK',
'AAA0149CEAF',
'AAA0149CDZE',
'AAA0149CDYN',
'AAA0149CCOE',
'AAA0149CCNN',
'AAA0149CCMS',
'AAA0149CCLW',
'AAA0149CDHK',
'AAA0044CHTD',
'AAA0044CHUH',
'AAA0044CHWW',
'AAA0044CHXS',
'AAA0044CHYN',
'AAA0044CJAW',
'AAA0044CJBS',
'AAA0044CJCN',
'AAA0044CJEP',
'AAA0044CJFZ',
'AAA0044CJHK',
'AAA0044CJKC',
'AAA0044CJLF',
'AAA0044CJMR',
'AAA0044CJNX',
'AAA0044CJPA',
'AAA0044CJRJ',
'AAA0044CJSY',
'AAA0044CJTD',
'AAA0044CJUH',
'AAA0044CJWW',
'AAA0044CJXS',
'AAA0044CJZE',
'AAA0044CKAF',
'AAA0044CKBR',
'AAA0044CKCX',
'AAA0044CKDM',
'AAA0042XDTO',
'AAA0042XETD',
'AAA0042XDOE',
'AAA0042XDLW',
'AAA0042XDKL',
'AAA0042XEYN',
'AAA0042XEZE',
'AAA0042XFAF',
'AAA0042XFBR',
'AAA0042XDDM',
'AAA0042XFDM',
'AAA0042XDCX',
'AAA0042XFLW',
'AAA0042XFJH',
'AAA0042XFFT',
'AAA0265JNZE',
'AAA0265JOAF',
'AAA0265JOBR',
'AAA0265JOCX',
'AAA0265JNHK',
'AAA0044CNBS',
'AAA0044CNAW',
'AAA0044CMZM',
'AAA0044CLZE',
'AAA0044CMAF',
'AAA0044CMBR',
'AAA0044CMWF',
'AAA0044CMCX',
'AAA0044CMTO',
'AAA0044CMEA',
'AAA0044CMSK',
'AAA0044CMFT',
'AAA0044CMRU',
'AAA0044CMPP',
'AAA0044CMJH',
'AAA0044CMOE',
'AAA0044CMNN',
'AAA0043MUWW',
'AAA0043MUTD',
'AAA0043MWSY',
'AAA0043MULF',
'AAA0043MUKC',
'AAA0043MUJZ',
'AAA0043MUFZ',
'AAA0043MUDE',
'AAA0043NOCX',
'AAA0043NOBR',
'AAA0043NPYN',
'AAA0043NPXS',
'AAA0043NOLW',
'AAA0039ZKZE',
'AAA0039ZKXS',
'AAA0039YTFZ',
'AAA0039YTDE',
'AAA0039YTCN',
'AAA0039YTKC',
'AAA0039YSWF',
'AAA0039YSUZ',
'AAA0038WLXR',
'AAA0040CZNX',
'AAA0040CZPA',
'AAA0040CZRJ',
'AAA0040CZSY',
'AAA0040DBKL',
'AAA0040DBJH',
'AAA0040DBHY',
'AAA0040DBFT',
'AAA0040DBEA',
'AAA0040DBDM',
'AAA0040DBCX',
'AAA0040DBBR',
'AAA0040CZTD',
'AAA0040DBAF',
'AAA0040CZUH',
'AAA0040DAZE',
'AAA0040DAYN',
'AAA0040DAWW',
'AAA0040DBLW',
'AAA0040DCJZ',
'AAA0040DCKC',
'AAA0040DCLF',
'AAA0040DCEP',
'AAA0040DCDE',
'AAA0011RLCX',
'AAA0011RJBR',
'AAA0011RJMS',
'AAA0238ARFT',
'AAA0238ARDM',
'AAA0238AREA',
'AAA0011RJKL',
'AAA0011RJEA',
'AAA0011RJFT',
'AAA0011RJJH',
'AAA0011REAW',
'AAA0011RDOE',
'AAA0011RDXR',
'AAA0011RDZM',
'AAA0011RDPP',
'AAA0011RDRU',
'AAA0011RDSK',
'AAA0011RDTO',
'AAA0011RDUZ',
'AAA0034XAFT',
'AAA0034XAEA',
'AAA0034XBCN',
'AAA0034WTKL',
'AAA0034WTJH',
'AAA0034WTXR',
'AAA0032RSNN',
'AAA0032RSOE',
'AAA0032RSPP',
'AAA0032RSRU',
'AAA0032RSSK',
'AAA0032RTCN',
'AAA0032RPWW',
'AAA0032RPZE',
'AAA0034RCEA',
'AAA0034RCFT',
'AAA0162TAAF',
'AAA0034RCHY',
'AAA0034RCJH',
'AAA0034RCKL',
'AAA0034RCLW',
'AAA0034RCMS',
'AAA0034RCNN',
'AAA0072LWCX',
'AAA0072LWEA',
'AAA0072LWHY',
'AAA0072LWJH',
'AAA0072LWKL',
'AAA0072LWLW',
'AAA0072LWMS',
'AAA0072LWNN',
'AAA0072LWOE',
'AAA0072LWPP',
'AAA0072LWRU',
'AAA0072LWSK',
'AAA0072LWTO',
'AAA0072LWUZ',
'AAA0072LWWF',
'AAA0072LWXR',
'AAA0188SANX',
'AAA0188SAOM',
'AAA0072LTAW',
'AAA0072LTCN',
'AAA0072LTNX',
'AAA0072LTOM',
'AAA0072LTRJ',
'AAA0072LTSY',
'AAA0072LTUH',
'AAA0072LTZE',
'AAA0088EWYX',
'AAA0088EWZM',
'AAA0088EXBS',
'AAA0083BJKC',
'AAA0223DNPA',
'AAA0223DOWF',
'AAA0223DOUZ',
'AAA0223DNTD',
'AAA0223DNWW',
'AAA0223DNXS',
'AAA0223DNZE',
'AAA0223DOAF',
'AAA0223DOBR',
'AAA0223DOXR',
'AAA0223DODM',
'AAA0223DOEA',
'AAA0223DOFT',
'AAA0223DOHY',
'AAA0223DOJH',
'AAA0223DOKL',
'AAA0223DOMS',
'AAA0223DONN',
'AAA0223DOOE',
'AAA0223DOSK',
'AAA0223DOCX',
'AAA0083BLHK',
'AAA0083BLKC',
'AAA0083BLLF',
'AAA0083BLOM',
'AAA0083BLRJ',
'AAA0083BLWW',
'AAA0083BLXS',
'AAA0083BLZE',
'AAA0083BMAF',
'AAA0083BMBR',
'AAA0083BMCX',
'AAA0083BMEA',
'AAA0083BMFT',
'AAA0083BMJH',
'AAA0083BMMS',
'AAA0083BMNN',
'AAA0083BMRU',
'AAA0083BMWF',
'AAA0220KJZE',
'AAA0220KJYN',
'AAA0220KFXS',
'AAA0220KJXS',
'AAA0220KJUH',
'AAA0220KHAW',
'AAA0220KHCN',
'AAA0220KHDE',
'AAA0220KHEP',
'AAA0220KHFZ',
'AAA0220KHHK',
'AAA0220KHJZ',
'AAA0220KHKC',
'AAA0220KHLF',
'AAA0220KHMR',
'AAA0220KHNX',
'AAA0220KHOM',
'AAA0220KHPA',
'AAA0220KHRJ',
'AAA0220KHSY',
'AAA0220KHTD',
'AAA0220KHWW',
'AAA0220KHXS',
'AAA0220KHYN',
'AAA0220KHZE',
'AAA0220KJAW',
'AAA0220KJBS',
'AAA0220KJDE',
'AAA0220KJEP',
'AAA0220KJHK',
'AAA0220KJJZ',
'AAA0220KJKC',
'AAA0220KJLF',
'AAA0220KJMR',
'AAA0220KJNX',
'AAA0220KJOM',
'AAA0220KJPA',
'AAA0220KJRJ',
'AAA0220KJSY',
'AAA0220KFZE',
'AAA0091CENN',
'AAA0091CEOE',
'AAA0091CEPP',
'AAA0091CERU',
'AAA0091CFDE',
'AAA0091CFHK',
'AAA0091CFJZ',
'AAA0091CBOM',
'AAA0091CBPA',
'AAA0091CBRJ',
'AAA0091CCAF',
'AAA0091CCBR',
'AAA0091CCCX',
'AAA0091CCDM',
'AAA0091CCEA',
'AAA0091CCFT',
'AAA0091CCJH',
'AAA0091CCKL',
'AAA0091CCLW',
'AAA0091CCMS',
'AAA0091CCNN',
'AAA0091CCOE',
'AAA0091CCPP',
'AAA0091CCRU',
'AAA0091CCSK',
'AAA0091CCTO',
'AAA0091CCUZ',
'AAA0091CCXR',
'AAA0091CCYX',
'AAA0091CCZM',
'AAA0091CDHK',
'AAA0091CDJZ',
'AAA0091CDKC',
'AAA0091CDLF',
'AAA0091CDMR',
'AAA0091CDPA',
'AAA0091CDRJ',
'AAA0091CDSY',
'AAA0091CDTD',
'AAA0175DZKC',
'AAA0083XZSY',
'AAA0083XYSK',
'AAA0163PXWF',
'AAA0163PXYX',
'AAA0163PXXR',
'AAA0163PXZM',
'AAA0094HNEP',
'AAA0094HNFZ',
'AAA0094HNHK',
'AAA0094HNJZ',
'AAA0094HMLW',
'AAA0094HDPA',
'AAA0094HDRJ',
'AAA0094HDSY',
'AAA0094HDTD',
'AAA0094HDUH',
'AAA0094HDXS',
'AAA0094HDZE',
'AAA0094HEAF',
'AAA0094HEBR',
'AAA0094HECX',
'AAA0094HEDM',
'AAA0094HEEA',
'AAA0094HEKL',
'AAA0094HELW',
'AAA0094HENN',
'AAA0094HEOE',
'AAA0094HESK',
'AAA0094HETO',
'AAA0094HEXR',
'AAA0094HEYX',
'AAA0094HEZM',
'AAA0094HFAW',
'AAA0094HFBS',
'AAA0094HFEP',
'AAA0094HFHK',
'AAA0094HFJZ',
'AAA0094HFKC',
'AAA0094HFMR',
'AAA0094HFNX',
'AAA0094HFOM',
'AAA0094HFRJ',
'AAA0094HFSY',
'AAA0086SMUZ',
'AAA0086SMWF',
'AAA0086SMYX',
'AAA0086SNUH',
'AAA0086SNWW',
'AAA0086SNXS',
'AAA0086SNYN',
'AAA0086SNZE',
'AAA0086SOAF',
'AAA0086SOBR',
'AAA0086SOCX',
'AAA0086SODM',
'AAA0086SOEA',
'AAA0086SOFT',
'AAA0011RJDM',
'AAA0034WTMS',
'AAA0034WTNN',
'AAA0040CZWW',
'AAA0040CZXS',
'AAA0040DCFZ',
'AAA0042XDBR',
'AAA0042XFCX',
'AAA0043MWWW',
'AAA0044CLXS',
'AAA0072LTKC',
'AAA0072LTWW',
'AAA0072LTXS',
'AAA0072LWDM',
'AAA0072LWFT',
'AAA0083BNAW',
'AAA0088EXAW',
'AAA0091CDFZ',
'AAA0094HDYN',
'AAA0094HEHY',
'AAA0094HEMS',
'AAA0094HFFZ',
'AAA0094HFUH',
'AAA0140DHNX',
'AAA0150HJMS',
'AAA0150HJOE',
'AAA0150HJPP',
'AAA0150HJRU',
'AAA0150HJSK',
'AAA0150HJUZ',
'AAA0150HJYX',
'AAA0150HJZM',
'AAA0150HKAW',
'AAA0150HKBS',
'AAA0150HKCN',
'AAA0150HKDE',
'AAA0150HKEP',
'AAA0150HKFZ',
'AAA0150HKHK',
'AAA0150HKJZ',
'AAA0150HKLF',
'AAA0150HKMR',
'AAA0150HKNX',
'AAA0150HKOM',
'AAA0150HKPA',
'AAA0150HKRJ',
'AAA0150HKSY',
'AAA0150HKTD',
'AAA0150HMBS',
'AAA0150HMCN',
'AAA0150HMDE',
'AAA0150HMEP',
'AAA0150HMFZ',
'AAA0150HMHK',
'AAA0150HMJZ',
'AAA0150HMKC',
'AAA0150HMLF',
'AAA0150HMMR',
'AAA0150HMNX',
'AAA0150HMOM',
'AAA0150HMPA',
'AAA0150HMRJ',
'AAA0150HMTD',
'AAA0150HMUH',
'AAA0150HMWW',
'AAA0150HMXS',
'AAA0150HMYN',
'AAA0150HMZE',
'AAA0150HNAF',
'AAA0150HNBR',
'AAA0150HNCX',
'AAA0150HNDM',
'AAA0150HNEA',
'AAA0188SAMR',
'AAA0188SARJ',
'AAA0188SATD',
'AAA0198HFPP',
'AAA0200XHUZ',
'AAA0202RWTD',
'AAA0202RWWW',
'AAA0202RWXS',
'AAA0202RWYN',
'AAA0223DNYN',
'AAA0223DOLW',
'AAA0254YRUZ',
'AAA0265JNJZ',
'AAA0265JNLF',
'AAA0265JNSY',
'AAA0265JNTD',
'AAA0265JNUH',
'AAA0265JNWW',
'AAA0265JNXS',
'AAA0265JNYN',
'AAA0011SZOM',
'AAA0041RUDM',
'AAA0011SZMR',
'AAA0011SZNX',
'AAA0011SZPA',
'AAA0011SZRJ',
'AAA0042PBKL',
'AAA0042PDJH',
'AAA0041RTYN',
'AAA0041RULW',
'AAA0042OYAF',
'AAA0042PDLW',
'AAA0011UAJH',
'AAA0011TZZM',
'AAA0042PCEP',
'AAA0041RTPA',
'AAA0042PCBS',
'AAA0042PDSK',
'AAA0012PWLW',
'AAA0041RWJH',
'AAA0041RUPP',
'AAA0041RTWW',
'AAA0217DWZE',
'AAA0012PWTO',
'AAA0042PDNN',
'AAA0012PXBS',
'AAA0012PWPP',
'AAA0041RUOE',
'AAA0041RUSK',
'AAA0011UABR',
'AAA0012PWXR',
'AAA0041RTRJ',
'AAA0012PWRU',
'AAA0042PBNN',
'AAA0041RTSY',
'AAA0012PXDE',
'AAA0012PXCN',
'AAA0164UYBS',
'AAA0012PWYX',
'AAA0041RTHK',
'AAA0041RTFZ',
'AAA0167MOUZ',
'AAA0041RUMS',
'AAA0042PCDE',
'AAA0041RTTD',
'AAA0041RTEP',
'AAA0011TBWF',
'AAA0041RTCN',
'AAA0011UAKL',
'AAA0012PWZM',
'AAA0012PWSK',
'AAA0011UALW',
'AAA0042PBHY',
'AAA0041RUKL',
'AAA0012PWOE',
'AAA0041RTXS',
'AAA0011UAAF',
'AAA0011TZXR',
'AAA0012PWMS',
'AAA0012PXAW',
'AAA0041RTDE',
'AAA0041RTAW',
'AAA0042PDKL',
'AAA0011TZWF',
'AAA0041RTKC',
'AAA0041RTMR',
'AAA0041RUXR',
'AAA0041RTLF',
'AAA0042PBRU',
'AAA0011SZSY',
'AAA0011TZYX',
'AAA0011TBUZ',
'AAA0041RTJZ',
'AAA0041RUZM',
'AAA0041RUYX',
'AAA0011TZSK',
'AAA0012PWNN',
'AAA0042PDRU',
'AAA0011TZPP',
'AAA0012PWKL',
'AAA0042OYDM',
'AAA0041RUEA',
'AAA0179OTXR',
'AAA0042PDOE',
'AAA0041RURU',
'AAA0012PWUZ',
'AAA0042PBPP',
'AAA0042PBLW',
'AAA0042PBXR',
'AAA0042PBOE',
'AAA0041RUUZ',
'AAA0041RTOM',
'AAA0266PSHK',
'AAA0041RTUH',
'AAA0265JNKC',
'AAA0220KHBS',
'AAA0220KJWW',
'AAA0091CBWW',
'AAA0220KJFZ',
'AAA0220KHUH',
'AAA0148ZBDE',
'AAA0044CJYN',
'AAA0148ZWPP',
'AAA0011RDYX',
'AAA0220KJCN',
'AAA0094HFLF',
'AAA0094HEFT',
'AAA0094HERU',
'AAA0072LTJZ',
'AAA0094HFCN',
'AAA0188SAPA',
'AAA0094HEUZ',
'AAA0094HNDE',
'AAA0094HNCN',
'AAA0044CJOM',
'AAA0094HEJH',
'AAA0091CBXS',
'AAA0034WTHY',
'AAA0034RCDM',
'AAA0162TABR',
'AAA0088EXCN',
'AAA0220KFYN',
'AAA0083BLTD',
'AAA0223DOPP',
'AAA0091CEAF',
'AAA0091CEBR',
'AAA0091CECX',
'AAA0223DOTO',
'AAA0091CDCN',
'AAA0091CDDE',
'AAA0044CMUZ',
'AAA0072LTLF',
'AAA0072LTMR',
'AAA0148ZAXR',
'AAA0083BLFZ',
'AAA0091CDEP',
'AAA0011RLBR',
'AAA0094HNNX',
'AAA0094HNOM',
'AAA0091CCWF',
'AAA0083BLDE',
'AAA0011RJLW',
'AAA0149DFRU',
'AAA0094HNPA',
'AAA0139RMZM',
'AAA0149DELF',
'AAA0149DETD',
'AAA0265JNNX',
'AAA0235MSAF',
'AAA0148ZUXR',
'AAA0149DFPP',
'AAA0149ADPA',
'AAA0265JNMR',
'AAA0148ZBKC',
'AAA0044CJDE',
'AAA0265JNOM',
'AAA0265JNPA',
'AAA0265JNRJ',
'AAA0044CLWW',
'AAA0044CMMS',
'AAA0044CFDE',
'AAA0040DBMS',
'AAA0040CZOM',
'AAA0011RDWF',
'AAA0088EWWF',
'AAA0091CBUH',
'AAA0083BLSY',
'AAA0083BMUZ',
'AAA0149DESY',
'AAA0094HNLF',
'AAA0083BMTO',
'AAA0083BMXR',
'AAA0034WTYX',
'AAA0148ZAWF',
'AAA0083BLMR',
'AAA0091CDWW',
'AAA0044CMLW',
'AAA0034RCCX',
'AAA0034RETO',
'AAA0034RESK',
'AAA0034RBYN',
'AAA0091CRNX',
'AAA0175DZLF',
'AAA0086SMZM',
'AAA0043MUMR',
'AAA0043MWNX',
'AAA0042XFKL',
'AAA0042XEUH',
'AAA0043MXCX',
'AAA0172WULW',
'AAA0043NOJH',
'AAA0091CBCN',
'AAA0034RCSK',
'AAA0043MXBR',
'AAA0042XFHY',
'AAA0043NODM',
'AAA0043MWXS',
'AAA0043MWUH',
'AAA0042XDEA',
'AAA0042XDJH',
'AAA0042XDNN',
'AAA0042XDWF',
'AAA0264XOCN',
'AAA0091CDUH',
'AAA0040DAXS',
'AAA0042XERJ',
'AAA0042XEMR',
'AAA0043MWZE',
'AAA0042XDRU',
'AAA0042XDUZ',
'AAA0088EXEP',
'AAA0044CMHY',
'AAA0043MWPA',
'AAA0042XDSK',
'AAA0042XEWW',
'AAA0042XEPA',
'AAA0094HFXS',
'AAA0043MUOM',
'AAA0042XDMS',
'AAA0083XZRJ',
'AAA0042XDFT',
'AAA0042XDXR',
'AAA0044CBLF',
'AAA0091CELW',
'AAA0034REKL',
'AAA0094HFPA',
'AAA0044CNFZ',
'AAA0043NOEA',
'AAA0039YTPA',
'AAA0039YTTD',
'AAA0044CLLF',
'AAA0044CMXR',
'AAA0044CNDE',
'AAA0044CNEP',
'AAA0044CLYN',
'AAA0044CLUH',
'AAA0044CLTD',
'AAA0039YSXR',
'AAA0275MOAF',
'AAA0042XDHY',
'AAA0038WMBS',
'AAA0039YTJZ',
'AAA0163CBEP',
'AAA0039YSYX',
'AAA0039YSZM',
'AAA0039YTBS',
'AAA0043MWMR',
'AAA0039YTSY',
'AAA0039YTMR',
'AAA0039YTLF',
'AAA0094HFYN',
'AAA0044CMYX',
'AAA0043MUNX',
'AAA0038TUKL',
'AAA0039YTNX',
'AAA0038WLYX',
'AAA0039ZLDM',
'AAA0032RSLW',
'AAA0034RATO',
'AAA0038WLTO',
'AAA0265XSNN',
'AAA0039ZLBR',
'AAA0088EZFZ',
'AAA0088EZCN',
'AAA0091CEKL',
'AAA0083XZJZ',
'AAA0242DZKL',
'AAA0034WTLW',
'AAA0011RDNN',
'AAA0034RERU',
'AAA0034RCAF',
'AAA0034RCBR',
'AAA0083BJHK',
'AAA0094HFWW',
'AAA0034RARU',
'AAA0034RASK',
'AAA0149CDCN',
'AAA0149CDOM',
'AAA0149CDDE',
'AAA0149CDEP',
'AAA0149CDFZ',
'AAA0149CETO',
'AAA0149CEUZ',
'AAA0149CCTO',
'AAA0149CEWF',
'AAA0149CCSK',
'AAA0149CEXR',
'AAA0149CCRU',
'AAA0149CEYX',
'AAA0149CCPP',
'AAA0149CECX',
'AAA0083BMHY',
'AAA0072LUHY',
'AAA0039YTUH',
'AAA0044CNCN',
'AAA0039ZKYN',
'AAA0034RBXS',
'AAA0094HFDE',
'AAA0091CCHY',
'AAA0094HDJZ',
'AAA0091CEJH',
'AAA0072LWZM',
'AAA0072LUEA',
'AAA0088EZHK',
'AAA0072LUOE',
'AAA0072LTBS',
'AAA0094HEWF',
'AAA0223DORU',
'AAA0091CDBS',
'AAA0088EXDE',
'AAA0223DNUH',
'AAA0188SASY',
'AAA0083BMLW',
'AAA0038WLWF',
'AAA0039YTAW',
'AAA0149CDLF',
'AAA0149CEDM',
'AAA0039YTOM',
'AAA0038WMDE',
'AAA0039ZLCX',
'AAA0083BSHY',
'AAA0083BSDM',
'AAA0083BSEA',
'AAA0083BSFT',
'AAA0043MUUH',
'AAA0088EWXR',
'AAA0083BSJH',
'AAA0149DEZE',
'AAA0149DENX',
'AAA0091CFKC',
'AAA0149CDXS',
'AAA0148ZURU',
'AAA0149DHTO',
'AAA0148ZUZM',
'AAA0094HNMR',
'AAA0094HNRJ',
'AAA0091CDAW',
'AAA0043MXAF',
'AAA0043NPTD',
'AAA0083XZNX',
'AAA0039ZLHY',
'AAA0043MWKC',
'AAA0043MWRJ',
'AAA0094HMZM',
'AAA0086SNTD',
'AAA0094HNAW',
'AAA0188ZDJZ',
'AAA0148ZUPP',
'AAA0083XYRU',
'AAA0149CEBR',
'AAA0043NPWW',
'AAA0086SOJH',
'AAA0230NAZM',
'AAA0011RJHY',
'AAA0086SMXR',
'AAA0034REHY',
'AAA0011TZTO',
'AAA0041RTBS',
'AAA0041RSZM',
'AAA0011UAHY',
'AAA0041RWKL',
'AAA0011TZRU',
'AAA0011UADM',
'AAA0011TZUZ',
'AAA0011UACX',
'AAA0011UAFT',
'AAA0042PDMS',
'AAA0042PDHY',
'AAA0042OYHY',
'AAA0261ZMLW',
'AAA0011TBXR',
'AAA0011UAEA',
'AAA0072LTHK',
'AAA0091CEZM',
'AAA0091CFAW',
'AAA0094HFTD',
'AAA0091CEWF',
'AAA0091CBZE',
'AAA0149ADRJ',
'AAA0083BLJZ',
'AAA0091CFBS',
'AAA0091CFCN',
'AAA0032RPXS',
'AAA0042XFEA',
'AAA0083XYPP',
'AAA0160FPOM',
'AAA0040DBNN',
'AAA0149CDNX',
'AAA0140DHMR',
'AAA0041RUFT',
'AAA0042PCHK',
'AAA0149DEJZ',
'AAA0042XEOM',
'AAA0044CHZE',
'AAA0044CJJZ',
'AAA0091CDOM',
'AAA0091CDNX',
'AAA0091CBYN',
'AAA0091CEYX',
'AAA0043NOAF',
'AAA0042XENX',
'AAA0072LTFZ',
'AAA0091TCSY',
'AAA0083BNDE',
'AAA0088EWSK',
'AAA0091CBEP',
'AAA0091CBBS',
'AAA0150HJNN',
'AAA0091CFFZ',
'AAA0072LTPA',
'AAA0072LUKL',
'AAA0148ZBJZ',
'AAA0148ZUUZ',
'AAA0149CCUZ',
'AAA0149CDMR',
'AAA0149CDPA',
'AAA0149CEEA',
'AAA0012PWWF',
'AAA0012NSBR',
'AAA0043MWYN',
'AAA0043NOFT',
'AAA0043NOHY',
'AAA0043NOKL',
'AAA0043MWOM',
'AAA0043NPUH',
'AAA0043MWTD',
'AAA0043MXDM',
'AAA0034JFPA',
'AAA0034JFMR',
'AAA0034JFNX',
'AAA0034JHAW',
'AAA0034JFRJ',
'AAA0034JFYN',
'AAA0034JHCN',
'AAA0034JHBS',
'AAA0034JCSK',
'AAA0034JFWW',
'AAA0034JCTO',
'AAA0255LHWF',
'AAA0034JECX',
'AAA0034JFXS',
'AAA0034JBRJ',
'AAA0034JFZE',
'AAA0034JJOM',
'AAA0255LJBR',
'AAA0034JFLF',
'AAA0034JASK',
'AAA0034JJRJ',
'AAA0034JARU',
'AAA0034JDPA',
'AAA0034JFUH',
'AAA0034JBKC',
'AAA0034JJSY',
'AAA0034JFSY',
'AAA0034JFTD',
'AAA0034JDNX',
'AAA0034JAHY',
'AAA0034JHJZ',
'AAA0072LUUZ',
'AAA0072LTDE',
'AAA0265BJNX',
'AAA0072LTEP',
'AAA0072LUDM',
'AAA0072LUNN',
'AAA0072LUSK',
'AAA0072LUWF',
'AAA0072LUMS',
'AAA0072LUTO',
'AAA0072LUPP',
'AAA0072LURU',
'AAA0072LUXR',
'AAA0072LTTD',
'AAA0072LUAF',
'AAA0072LUCX',
'AAA0044CFCN',
'AAA0044CFEP',
'AAA0072LUJH',
'AAA0034REJH',
'AAA0034RCRU',
'AAA0038WDRU',
'AAA0038WMAW',
'AAA0154RXUZ',
'AAA0154RXSK',
'AAA0236NREP',
'AAA0038WDTO',
'AAA0038WDKL',
'AAA0038WDNN',
'AAA0038WLZM',
'AAA0038WMEP',
'AAA0236NRDE',
'AAA0038WDJH',
'AAA0038WDMS',
'AAA0038WDSK',
'AAA0094HMNN',
'AAA0011TBTO',
'AAA0094HDWW',
'AAA0094HMPP',
'AAA0094HMRU',
'AAA0094HMWF',
'AAA0094HMYX',
'AAA0094HMMS',
'AAA0094HMXR',
'AAA0094HNKC',
'AAA0094HNYN',
'AAA0094HNSY',
'AAA0094HNXS',
'AAA0240DKHK',
'AAA0094HNBS',
'AAA0203FNJZ',
'AAA0094HEPP',
'AAA0034JFOM',
'AAA0034JBMR',
'AAA0034JBSY',
'AAA0255LHYX',
'AAA0034JDTD',
'AAA0034JCDM',
'AAA0034JAWF',
'AAA0034JCUZ',
'AAA0034JBZE',
'AAA0034JBJZ',
'AAA0034JABR',
'AAA0034JDEP',
'AAA0034JBDE',
'AAA0034JFAW',
'AAA0034JALW',
'AAA0034JDJZ',
'AAA0034JEKL',
'AAA0034JBBS',
'AAA0034JAYX',
'AAA0034JEHY',
'AAA0034JDKC',
'AAA0034JESK',
'AAA0034JCPP',
'AAA0255LJCX',
'AAA0034JDRJ',
'AAA0034JCBR',
'AAA0034JBAW',
'AAA0034JDBS',
'AAA0255LHZM',
'AAA0034JDUH',
'AAA0034JBUH',
'AAA0034JEYX',
'AAA0034JDSY',
'AAA0034JDDE',
'AAA0034JETO',
'AAA0034JBXS',
'AAA0034JEZM',
'AAA0034HZOM',
'AAA0034JBPA',
'AAA0034HZYN',
'AAA0034JCHY',
'AAA0034JAAF',
'AAA0034JBHK',
'AAA0034JDLF',
'AAA0034JDZE',
'AAA0034JEAF',
'AAA0034JCAF',
'AAA0034JCKL',
'AAA0034JATO',
'AAA0034JHEP',
'AAA0034JHSY',
'AAA0034JHOM',
'AAA0034JEPP',
'AAA0034JEFT',
'AAA0034JCZM',
'AAA0034JCLW',
'AAA0034JDAW',
'AAA0034JCJH',
'AAA0034HZNX',
'AAA0034JCCX',
'AAA0034JEUZ',
'AAA0034JANN',
'AAA0034JAOE',
'AAA0255LJAF',
'AAA0034JAJH',
'AAA0034JEXR',
'AAA0034JBLF',
'AAA0034JELW',
'AAA0034JHNX',
'AAA0034JEBR',
'AAA0034JCRU',
'AAA0034JEWF',
'AAA0034JAKL',
'AAA0034JJMR',
'AAA0034JJUH',
'AAA0034JFDE',
'AAA0034JFFZ',
'AAA0034JDFZ',
'AAA0034JDMR',
'AAA0034JEDM',
'AAA0034JBWW',
'AAA0034JBYN',
'AAA0034JAZM',
'AAA0034JDHK',
'AAA0255LHTO',
'AAA0034JDYN',
'AAA0255LHUZ',
'AAA0034JBNX',
'AAA0034JEMS',
'AAA0034JEEA',
'AAA0034JBOM',
'AAA0034JAMS',
'AAA0034JAPP',
'AAA0034JDOM',
'AAA0034JCOE',
'AAA0034JCFT',
'AAA0034JDWW',
'AAA0034JAUZ',
'AAA0034JCXR',
'AAA0034JCYX',
'AAA0034JBEP',
'AAA0034JCNN',
'AAA0034JFCN',
'AAA0034JAXR',
'AAA0034JBTD',
'AAA0034JFEP',
'AAA0034JEJH',
'AAA0034JBFZ',
'AAA0034JDCN',
'AAA0255LHXR',
'AAA0034HZZE',
'AAA0032RSXR',
'AAA0034RAPP',
'AAA0034RELW',
'AAA0032RPYN',
'AAA0034RENN',
'AAA0032RPSY',
'AAA0032RPTD',
'AAA0032RPUH',
'AAA0032RSMS',
'AAA0034RCPP',
'AAA0032RSWF',
'AAA0271YEMS',
'AAA0034YJFT',
'AAA0034YJJH',
'AAA0034YJMS',
'AAA0034YJNN',
'AAA0034ZNTD',
'AAA0035AAJH',
'AAA0271YELW',
'AAA0034YSEP',
'AAA0034YSFZ',
'AAA0034ZNWW',
'AAA0034ZNXS',
'AAA0034ZNYN',
'AAA0034ZOCX',
'AAA0271YEOE',
'AAA0271YEPP',
'AAA0197NOLW',
'AAA0034YJHY',
'AAA0035AFHK',
'AAA0271YEJH',
'AAA0271YEKL',
'AAA0035AERU',
'AAA0035AESK',
'AAA0035AEWF',
'AAA0035AEXR',
'AAA0035AEYX',
'AAA0035AFAW',
'AAA0035AFBS',
'AAA0271YEFT',
'AAA0035AFCN',
'AAA0035AFDE',
'AAA0035AFEP',
'AAA0278UTFT',
'AAA0034ZFCN',
'AAA0034ZFEP',
'AAA0034ZFFZ',
'AAA0278UTLW',
'AAA0034ZXEP',
'AAA0034ZXFZ',
'AAA0034YJLW',
'AAA0271YENN',
'AAA0034YRZM',
'AAA0034ZXJZ',
'AAA0278UTJH',
'AAA0278UTHY',
'AAA0034ZZJZ',
'AAA0034RCOE',
'AAA0034RAUZ',
'AAA0034RBZE',
'AAA0034REMS',
'AAA0034REPP',
'AAA0034WTOE',
'AAA0034REUZ',
'AAA0038WDLW',
'AAA0042XCMR',
'AAA0042XEBS',
'AAA0042XDPP',
'AAA0042XEXS',
'AAA0042PBSK',
'AAA0271MLNX',
'AAA0271MLLF',
'AAA0271MLKC',
'AAA0271MLMR',
'AAA0271MLJZ',
'AAA0089WMTD',
'AAA0083XZMR',
'AAA0083XZOM',
'AAA0083XZLF',
'AAA0083XZPA',
'AAA0150HJTO',
'AAA0030NWUZ',
'AAA0083BLUH',
'AAA0083BMKL',
'AAA0223DNRJ',
'AAA0083BNCN',
'AAA0083BLNX',
'AAA0083BLYN',
'AAA0083BMOE',
'AAA0083BLPA',
'AAA0083BMPP',
'AAA0083BMSK',
'AAA0220KJTD',
'AAA0083BMDM',
'AAA0083BSCX',
'AAA0083XSZM',
'AAA0083BNBS',
'AAA0083BLEP',
'AAA0223DNSY',
'AAA0083BSBR',
'AAA0091TCZE',
'AAA0091TCWW',
'AAA0091TDBR',
'AAA0091TCRJ',
'AAA0091CBSY',
'AAA0091TCPA',
'AAA0091CBAW',
'AAA0091TCYN',
'AAA0091CEHY',
'AAA0091CETO',
'AAA0091TCOM',
'AAA0091TCTD',
'AAA0091CBHK',
'AAA0091CESK',
'AAA0091CEUZ',
'AAA0091TCUH',
'AAA0091TCXS',
'AAA0091CEXR',
'AAA0086SMMS',
'AAA0086SOKL',
'AAA0086SMOE',
'AAA0086SMSK',
'AAA0086SMRU',
'AAA0086SMTO',
'AAA0086SNSY',
'AAA0253WSHY',
'AAA0253WSDM',
'AAA0086SMNN',
'AAA0088EWUZ',
'AAA0039YTRJ',
'AAA0039YSTO',
'AAA0040DFFT',
'AAA0040DFEA',
'AAA0040DFJH',
'AAA0040DFHY',
'AAA0040CZKC',
'AAA0040CZLF',
'AAA0040CZMR',
'AAA0040CZJZ',
'AAA0088EZJZ',
'AAA0042JCFZ',
'AAA0042JCHK',
'AAA0042JCJZ',
'AAA0088EWTO',
'AAA0041RUJH',
'AAA0268YZXS',
'AAA0235ASUZ',
'AAA0091CEMS',
'AAA0091TDAF',
'AAA0044CMKL',
'AAA0265JODM',
'AAA0044CMDM',
'AAA0278UTKL',
'AAA0085PHLF',
'AAA0086SNBS',
'AAA0086SNCN',]

In [0]:
lista_consulta = ['50S-40113684',
'50S-40108906',
'50S-40113673',
'50S-40151379',
'50S-40140942',
'50S-40151382',
'50S-40056348',
'50S-40056351',
'50S-40056352',
'50S-40056353',
'50S-40056354',
'50S-40056359',
'50S-40056358',
'50S-40056357',
'50S-40056355',
'50S-40056377',
'50S-40056378',
'50S-40056380',
'50S-40056388',
'50S-40056387',
'50S-40056386',
'50S-40056385',
'50S-40126961',
'50S-40056382',
'50S-40056411',
'50S-40056414',
'50S-40056415',
'50S-40162836',
'50S-40136922',
'50S-40186199',
'50S-40082048',
'50S-40172659',
'50S-40074574',
'50S-40108908',
'50S-40141257',
'50S-40144541',
'50S-40147635',
'50S-40056420',
'50S-40056419',
'50S-40056418',
'50S-40056439',
'50S-40056438',
'50S-40056437',
'50S-40056436',
'50S-40056451',
'50S-266908',
'50S-266909',
'50S-266910',
'50S-266911',
'50S-266912',
'50S-266914',
'50S-266915',
'50S-266916',
'50S-266918',
'50S-266919',
'50S-266920',
'50S-266922',
'50S-266923',
'50S-267002',
'50S-267001',
'50S-266999',
'50S-266998',
'50S-266997',
'50S-266996',
'50S-266995',
'50S-266994',
'50S-266993',
'50S-266991',
'50S-266990',
'50S-266989',
'50S-266988',
'50S-266987',
'050S00372515',
'50S-144540',
'50S-126735',
'50S-853637',
'50S-1033789',
'50S-72538',
'50S-1002310',
'50S-197599',
'50S-228576',
'50S-898257',
'50S-40192789',
'50S-668881',
'50S-40268594',
'50S-516111',
'50S-360494',
'50S-40745734',
'50S-40745735',
'50S-40745736',
'50S-40745737',
'50S-40745739',
'50S-474680',
'50S-689447',
'50S-75700',
'50S-11010',
'50S-944358',
'50S-40336383',
'50S-554236',
'50S-40125730',
'50S-1038595',
'50S-573757',
'50S-40022373',
'50S-925202',
'50S-718676',
'50S-115882',
'50S-434019',
'50S-40056024',
'50S-562738',
'50S-339080',
'50S-598588',
'50S-40017268',
'50S-40108154',
'50S-136818',
'50S-538440',
'50S-498306',
'50S-658935',
'50S-40157062',
'50S-128324',
'50S-561620',
'50S-40258267',
'50S-708747',
'50S-472807',
'50S-277560',
'50S-33490',
'50S-40141653',
'50S-157127',
'50S-40639837',
'50S-40209163',
'50S-40086434',
'50S-34730',
'50S-120515',
'50S-5487',
'50S-29815',
'50S-29816',
'50S-14098',
'50S-298424',
'50S-152700',
'50S-41028',
'50S-90226',
'50S-687813',
'50S-766193',
'50S-90211',
'50S-97027',
'50S-97026',
'50S-97029',
'50S-97028',
'50S-97030',
'50S-97034',
'50S-40197423',
'50S-40136980',
'50S-40136981',
'50S-40136982',
'50S-777320',
'50S-913962',
'050C00457663',
'50S-592262',
'50S-40349892',
'50S-40557176',
'050S40557178',
'050S40557177',
'50S-971400',
'50C-207701',
'50S-44158',
'50S-616112',
'50S-118442',
'50S-430787',
'50S-821815',
'50S-821817',
'50S-57204',
'50S-40135068',
'50C-244645',
'50S-550538',
'50S-40459916',
'50C-378678',
'50C-912105',
'50C-883922',
'50C-1250751',
'50C-1441048',
'50C-307205',
'50C-1280039',
'50C-1280040',
'50C-1280041',
'50C-1280042',
'50C-1280043',
'50C-1280044',
'50C-18106',
'50C-1468510',
'50C-637979',
'50C-637986',
'50C-1519706',
'50C-637982',
'50C-637980',
'50C-637983',
'50C-637981',
'50C-637985',
'50C-637984',
'50C-344860',
'50C-344862',
'50C-344864',
'50C-344865',
'50C-344866',
'50C-344867',
'50C-344868',
'50C-344869',
'50C-344870',
'50C-344871',
'50C-344872',
'50C-344873',
'50C-344874',
'50C-344875',
'50C-344876',
'50C-344877',
'50C-1643685',
'50C-1643627',
'50C-1169235',
'50C-1169237',
'50C-1169243',
'50C-1169244',
'50C-88557',
'50C-88558',
'50C-88560',
'50C-88564',
'50C-66522',
'50C-66518',
'50C-66517',
'50C-622832',
'50C-1669182',
'50C-1669183',
'50C-1669184',
'50C-1669187',
'50C-1669189',
'50C-1669190',
'50C-1669192',
'50C-1669193',
'50C-1669194',
'50C-1669195',
'50C-1669196',
'50C-1669197',
'50C-1669198',
'50C-1669199',
'50C-1669200',
'50C-1669201',
'50C-1669203',
'50C-1669204',
'50C-1669205',
'50C-1669208',
'50C-1669210',
'50C-1293463',
'50C-1293465',
'50C-1293466',
'50C-1293469',
'50C-1293471',
'50C-1293475',
'050C01293476',
'50C-1293478',
'50C-1293479',
'50C-1293480',
'50C-1293481',
'50C-1293483',
'50C-1293484',
'50C-1293486',
'50C-1293489',
'50C-1293490',
'50C-1293492',
'50C-1293496',
'50C-1775244',
'50C-1775245',
'50C-1775246',
'50C-1775247',
'50C-1775249',
'50C-1775250',
'50C-1775252',
'50C-1775253',
'50C-1775256',
'50C-1775257',
'50C-1775258',
'50C-1775259',
'50C-1775260',
'50C-1775261',
'50C-1775215',
'50C-1775216',
'50C-1775217',
'50C-1775218',
'50C-1775219',
'50C-1775220',
'50C-1775221',
'50C-1775223',
'50C-1775224',
'50C-1775225',
'50C-1775226',
'50C-1775227',
'50C-1775228',
'50C-1775230',
'50C-1775232',
'50C-1775234',
'50C-1775235',
'50C-1775236',
'50C-1775237',
'50C-1775238',
'50C-1775239',
'50C-1775240',
'50C-1775241',
'50C-1775242',
'50C-1775243',
'50C-1775255',
'50C-616473',
'50C-613540',
'50C-616145',
'50C-616478',
'50C-1249181',
'50C-791103',
'50C-791104',
'50C-163322',
'50C-163323',
'50C-163321',
'50C-853999',
'50C-854000',
'50C-854001',
'50C-854002',
'50C-854003',
'50C-854004',
'50C-1255019',
'50C-771114',
'50C-1227617',
'50C-415486',
'50C-890505',
'50C-1161301',
'50C-144216',
'50C-135502',
'50C-41353',
'50C-101171',
'50C-31065',
'50C-784026',
'50C-785190',
'50C-788480',
'50C-570459',
'50C-570460',
'50C-570461',
'50C-570462',
'50C-570463',
'50C-570466',
'50C-570467',
'50C-570468',
'50C-570469',
'50C-1205067',
'50C-1372483',
'50C-157345',
'50C-1525372',
'50C-1525373',
'50C-1525374',
'50C-1525375',
'50C-706207',
'50C-706208',
'50C-706209',
'50C-706210',
'50C-515529',
'50C-549873',
'50C-549874',
'50C-549875',
'50C-549876',
'50C-549877',
'50C-549879',
'50C-549881',
'50C-549882',
'50C-549883',
'50C-549884',
'50C-549885',
'050C00549886',
'50C-549890',
'50C-549891',
'50C-549893',
'50C-549894',
'50C-549897',
'50C-549898',
'50C-549901',
'50C-549902',
'50C-549903',
'50C-549904',
'50C-549905',
'50C-549908',
'50C-549910',
'50C-549911',
'50C-549912',
'50C-549870',
'50C-549871',
'50C-549872',
'50C-291047',
'50C-290488',
'50C-338138',
'50C-69176',
'050C-01240456',
'50C-486810',
'50C-486811',
'50C-486812',
'50C-486813',
'50C-486814',
'50C-486815',
'50C-486816',
'50C-486806',
'50C-486807',
'50C-486808',
'50C-486809',
'50S-899252',
'50C-465026',
'050C01457046',
'50S-97031',
'50S-97033',
'50S-882880',
'50S-440490',
'50S-40032587',
'50S-493834',
'50S-40416316',
'50C-1169240',
'50C-88561',
'50C-88562',
'50C-344861',
'50C-344863',
'50C-428555',
'50C-66521',
'50C-570458',
'50C-549880',
'50C-549888',
'50C-549892',
'50C-549909',
'50C-290481',
'50S-40161479',
'50S-40092394',
'50S-40092400',
'50S-40092402',
'50S-40092404',
'50S-40092406',
'50S-40092410',
'50S-40092416',
'50S-40092418',
'50S-40092419',
'50S-40092421',
'50S-40092420',
'50S-40092417',
'50S-40092415',
'50S-40092413',
'50S-40092411',
'50S-40092409',
'50S-40092405',
'50S-40092403',
'50S-40092401',
'50S-40225551',
'50S-40224710',
'50S-40092397',
'50S-40092396',
'50S-40092395',
'50S-40092450',
'50S-40092451',
'50S-40092452',
'50S-40092453',
'50S-40092454',
'50S-40092455',
'50S-40092456',
'50S-40092457',
'50S-40092458',
'50S-40092459',
'50S-40092460',
'50S-40092461',
'50S-40092462',
'50S-40092463',
'50S-40092464',
'50S-40092465',
'50S-40092466',
'50S-40092467',
'50S-40092468',
'50S-40092469',
'50S-40092470',
'50S-40092471',
'50S-40092472',
'50S-40092473',
'50S-40092474',
'50C-1643622',
'50C-1643624',
'50C-1643628',
'50S-40496791',
'50S-40496809',
'50S-40503140',
'50S-40503119',
'50S-40505071',
'50S-40503117',
'50C-1669191',
'50C-1669202',
'50S-40389166',
'50S-40745740',
'50S-40745742',
'50S-40745728',
'50S-40745729',
'50S-40745730',
'50S-40745731',
'50S-40745732',
'50S-40745733',
'50S-40200231',
'50S-40172725',
'50S-40200229',
'50S-40200230',
'50S-40200232',
'50S-40200233',
'50S-40473780',
'50S-40236717',
'50S-591222',
'50S-76008',
'50S-40241140',
'50S-40233732',
'50S-601824',
'50S-694956',
'50S-40319667',
'50S-823114',
'050S40425963',
'50S-40311929',
'50S-173304',
'50S-869439',
'50S-259290',
'50S-682560',
'50S-40539852',
'50S-182191',
'50S-40297871',
'50S-97193',
'50S-148597',
'50S-259289',
'50S-65775',
'50S-177756',
'50S-587532',
'50S-246993',
'50S-40234314',
'50S-587524',
'50S-200139',
'50S-89010',
'50S-40398470',
'50S-179842',
'50S-682551',
'50S-682552',
'50S-40352247',
'50S-720180',
'50S-40294308',
'50S-682555',
'50S-741587',
'50S-286794',
'50S-682550',
'50S-315405',
'50S-177759',
'50S-200136',
'50S-8801',
'50S-40241139',
'050S40058970',
'50S-182192',
'50S-682559',
'50S-347115',
'50S-835017',
'50S-182194',
'50S-193756',
'50S-742332',
'50S-742333',
'50S-40282895',
'50S-243082',
'50S-795958',
'50S-699397',
'50S-259292',
'50S-699396',
'50S-40430451',
'50S-40200234',
'50S-213227',
'50S-656757',
'50S-795956',
'50S-40109190',
'50S-259293',
'50S-508289',
'50S-182196',
'50S-40246405',
'50S-238639',
'50S-150378',
'50S-40247399',
'50S-40309933',
'50S-40378580',
'50S-40311343',
'50S-65774',
'50S-261348',
'50S-40247450',
'50S-40233731',
'50S-40333100',
'50S-40280365',
'50S-259291',
'50S-823113',
'50S-40751256',
'50S-00022547',
'50S-682556',
'50S-40745741',
'50C-1775251',
'50C-1775248',
'50C-752243',
'50C-1775233',
'50C-1775222',
'50S-40056356',
'50S-266992',
'50S-40126960',
'50S-821816',
'50C-1775229',
'50C-549869',
'50C-549887',
'50C-549896',
'50C-1169239',
'50C-549906',
'50C-1643626',
'50C-549899',
'50C-706206',
'50C-706205',
'50S-267000',
'50C-549889',
'50C-752244',
'50C-236890',
'50C-637978',
'50C-1519707',
'50C-66520',
'50C-1775231',
'50C-1293473',
'50C-1669206',
'50C-470558',
'50C-470559',
'50C-470560',
'50C-1669209',
'50C-1424393',
'50C-1424394',
'50S-913965',
'50C-1169241',
'50C-1169242',
'50S-40056350',
'50C-1293462',
'50C-570457',
'50S-372534',
'50C-585574',
'50C-585575',
'50C-783599',
'50C-1293460',
'50S-1190297',
'50S-40082039',
'50C-585576',
'50S-40170283',
'50S-40113686',
'50S-40188654',
'50S-40745744',
'050S40448835',
'50S-40056381',
'50S-40139534',
'50S-40056413',
'50S-40745743',
'50S-40056361',
'50S-266917',
'50S-40745745',
'50S-40745746',
'50S-40745747',
'50S-40151339',
'50S-1072667',
'050S00103907',
'50S-205129',
'50S-263288',
'50S-12419',
'50C-66519',
'050C00752242',
'50C-1293472',
'50C-1293495',
'050S40102270',
'50C-585572',
'50C-1293494',
'50C-1293497',
'50C-435649',
'50S-40056349',
'50C-1293467',
'50C-470554',
'50S-699276',
'50C-496731',
'50C-246822',
'50C-1220092',
'50C-328911',
'050C00281538',
'50C-1205066',
'050C01195400',
'50S-1118091',
'50S-513006',
'50S-54574',
'50S-347617',
'50S-40001155',
'50S-40299532',
'50S-40162012',
'50C-767504',
'50C-644108',
'50S-40305566',
'50S-356320',
'50S-40229203',
'50S-515474',
'50S-909689',
'50S-159771',
'50S-519454',
'50S-302443',
'50S-430855',
'50C-1587878',
'50C-542285',
'50S-97032',
'50S-456313',
'50S-244497',
'50S-664552',
'50S-40181896',
'50S-169822',
'50C-644919',
'50S-40328607',
'50S-40103126',
'50S-329819',
'50S-891023',
'50S-244408',
'50C-125676',
'50S-396946',
'50S-1165035',
'50C-996096',
'50S-40003693',
'50S-214358',
'50S-211878',
'50C-696802',
'50C-757829',
'50C-1362861',
'50S-54736',
'50S-158586',
'50S-3196',
'50S-757843',
'50S-213472',
'50S-26662',
'50S-40122266',
'50S-306549',
'50S-692843',
'50S-40098641',
'50S-515887',
'50S-111933',
'50S-12161',
'50S-1169781',
'50S-300627',
'50S-40265491',
'50S-40021364',
'50S-128401',
'50S-121475',
'50S-680445',
'50S-275260',
'50S-99695',
'50S-408937',
'50S-210543',
'50C-1202994',
'50S-1012813',
'50S-40520793',
'050S00920954',
'50S-57419',
'50S-1003450',
'50S-107601',
'50C-364500',
'50C-500628',
'50S-1067996',
'50S-546909',
'50S-742303',
'50C-42517',
'50C-320837',
'50C-261767',
'50C-83365',
'050S40660585',
'50C-242933',
'50S-34196',
'50C-317627',
'50C-1460596',
'50C-1460597',
'50C-658623',
'50C-542395',
'50C-281182',
'50C-41577',
'50S-40056447',
'50S-40056458',
'50S-40056448',
'50S-40056449',
'50S-40056450',
'50S-40056430',
'50S-40056431',
'50S-40056443',
'50S-40056432',
'50S-40056442',
'50S-40056433',
'50S-40056441',
'50S-40056434',
'50S-40056440',
'50S-40056422',
'50C-1293485',
'50C-648458',
'50S-341190',
'50S-323880',
'50S-991594',
'50C-103081',
'50C-549907',
'50C-1365077',
'50C-137912',
'50C-1508644',
'50C-322427',
'50C-1081905',
'50C-905008',
'50C-154007',
'50C-1169236',
'50C-549900',
'50C-1669207',
'50C-1424392',
'50C-66516',
'50C-1669188',
'50C-1643625',
'50C-1293488',
'50S-40055514',
'50S-40271237',
'50S-40056455',
'50S-40301085',
'50S-7302',
'50S-624216',
'50S-510190',
'50C-1344267',
'50C-91288',
'50C-157749',
'50C-1344265',
'50S-523169',
'50C-66523',
'50C-1352802',
'50S-40270995',
'50S-40229820',
'50C-791105',
'50S-40056417',
'50S-40056376',
'50S-40214368',
'50S-40056383',
'50C-585573',
'50C-586352',
'50C-25350',
'50S-667543',
'50S-365155',
'50C-996092',
'50S-502752',
'50S-454415',
'50S-40344250',
'50C-562967',
'50C-1426366',
'50C-433823',
'50C-1639148',
'50S-40056375',
'50C-1281566',
'50S-40056421',
'50S-48065',
'50C-1230603',
'50C-1580573',
'50C-646324',
'50C-467579',
'50C-409825',
'50S-160059',
'50S-682549',
'50S-741588',
'50S-40056051',
'50S-869441',
'50S-40051313',
'50C-516206',
'50S-161696',
'50S-398422',
'50S-7188',
'50S-40297872',
'50S-40283132',
'50S-40254309',
'50S-40742489',
'50S-00729900',
'50S-44026',
'50C-1169234',
'50C-1225162',
'50C-1225163',
'50C-288972',
'50C-1225156',
'50C-853998',
'50S-40056412',
'50C-1293464',
'50C-1225164',
'50C-1225165',
'50C-343981',
'50S-760426',
'50C-996542',
'50S-40382085',
'50S-25201',
'50S-40056457',
'50S-40196881',
'50S-635646',
'50S-40244653',
'50S-40135346',
'50S-683610',
'50S-266913',
'50S-266921',
'50C-570465',
'50C-570464',
'50C-752245',
'50C-1225161',
'50S-40489375',
'50S-274317',
'50C-1169233',
'50C-61482',
'50C-984272',
'50C-1036291',
'50C-1543591',
'50C-359330',
'50S-40092398',
'50C-791102',
'50C-473639',
'50C-725683',]